In [1]:
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.models import Model, Sequential

Using TensorFlow backend.


In [2]:
df_train = pd.read_csv('data/train.csv')

X_train = pd.read_csv('train_with_prob.csv', index_col=False)
X_train = X_train.drop('Unnamed: 0', axis=1)
X_train.head()

y_train = df_train['is_duplicate'].values

In [3]:
# Up/down sampling
pos_train = X_train[y_train == 1]
neg_train = X_train[y_train == 0]
X_train = pd.concat((neg_train, pos_train.iloc[:int(0.8 * len(pos_train))],
                     neg_train))
y_train = np.array([0] * neg_train.shape[0] + [1] * pos_train.iloc[:int(
    0.8 * len(pos_train))].shape[0] + [0] * neg_train.shape[0])
print(np.mean(y_train))
del pos_train, neg_train

0.189701079013


In [4]:
X_train.replace(np.inf, 99)
X_train.replace(np.nan, 0)

,word_match,tfidf_wm,tfidf_wm_stops,jaccard,wc_diff,wc_ratio,wc_diff_unique,wc_ratio_unique,wc_diff_unq_stop,wc_ratio_unique_stop,...,clf_123,clf_124,clf_125,clf_126,clf_127,clf_128,clf_129,clf_130,clf_131,clf_132
0,0.727273,0.796024,0.772164,0.769231,2,0.857143,1,0.916667,1,0.833333,...,0.515520,0.520313,0.494767,0.641264,0.520313,0.494767,0.641264,0.553826,0.522829,0.442324
1,0.307692,0.359200,0.361758,0.250000,5,1.625000,4,1.500000,5,2.250000,...,0.040380,0.266175,0.099237,0.176471,0.266175,0.099237,0.176471,0.294340,0.142857,0.491857
2,0.363636,0.302555,0.355191,0.200000,4,0.714286,4,0.714286,1,0.833333,...,0.246365,0.266175,0.351983,0.292521,0.266175,0.351983,0.292521,0.356969,0.237123,0.352159
3,0.000000,0.000000,0.000000,0.000000,2,0.818182,1,0.900000,1,1.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.034253,0.000000,0.111111,6,0.538462,6,0.538462,5,0.500000,...,0.000454,0.000241,0.000000,0.000000,0.000241,0.000000,0.000000,0.000214,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,7,2.750000,6,2.500000,5,3.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.500000,0.476358,0.396755,0.600000,0,1.000000,0,1.000000,0,1.000000,...,0.172575,0.573805,0.607670,0.083564,0.573805,0.607670,0.083564,0.196632,0.631554,0.299157
9,0.363636,0.483158,0.503203,0.200000,0,1.000000,0,1.000000,1,0.833333,...,0.119724,0.220930,0.115230,0.318435,0.220930,0.115230,0.318435,0.324381,0.192868,0.130705
10,0.000000,0.009127,0.000000,0.041667,10,2.111111,7,1.777778,0,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
14,0.818182,0.827309,0.804625,0.833333,0,1.000000,0,1.000000,0,1.000000,...,0.941176,0.080537,0.000000,0.348485,0.080537,0.000000,0.348485,0.019963,0.066510,0.189219


In [5]:
X_train['q1_q2_freq_diff'] = np.abs(X_train['q1_freq'] - X_train['q2_freq'])
X_train['q1_q2_freq_sum'] = X_train['q1_freq'] + X_train['q2_freq']
X_train['skew_diff'] = np.abs(X_train['skew_q1vec'] - X_train['skew_q2vec'])
X_train['skew_sum'] = X_train['skew_q1vec'] + X_train['skew_q2vec']

In [6]:
X_train.shape

(629464, 180)

In [7]:
X_train.head()

,word_match,tfidf_wm,tfidf_wm_stops,jaccard,wc_diff,wc_ratio,wc_diff_unique,wc_ratio_unique,wc_diff_unq_stop,wc_ratio_unique_stop,...,clf_127,clf_128,clf_129,clf_130,clf_131,clf_132,q1_q2_freq_diff,q1_q2_freq_sum,skew_diff,skew_sum
0,0.727273,0.796024,0.772164,0.769231,2,0.857143,1,0.916667,1,0.833333,...,0.520313,0.494767,0.641264,0.553826,0.522829,0.442324,1,3,0.123719,-0.060084
1,0.307692,0.359200,0.361758,0.250000,5,1.625000,4,1.500000,5,2.250000,...,0.266175,0.099237,0.176471,0.294340,0.142857,0.491857,5,11,0.085969,0.103438
2,0.363636,0.302555,0.355191,0.200000,4,0.714286,4,0.714286,1,0.833333,...,0.266175,0.351983,0.292521,0.356969,0.237123,0.352159,1,3,0.095198,0.384306
3,0.000000,0.000000,0.000000,0.000000,2,0.818182,1,0.900000,1,1.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,2,0.072176,0.067122
4,0.000000,0.034253,0.000000,0.111111,6,0.538462,6,0.538462,5,0.500000,...,0.000241,0.000000,0.000000,0.000214,0.000000,0.000000,2,4,0.248626,-0.019072


In [8]:
# np.unique(y_train[255000:256000])

In [9]:
# X_train, y_train = X_train[255000:256000], y_train[255000:256000]

In [10]:
# Initialize the constructor
model = Sequential()

# Add an input layer 
model.add(Dense(512, activation='relu', input_shape=(180,)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
# Add one hidden layer 
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
# Add an output layer 
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
# model.fit(data, labels)  # starts training

In [11]:
tr = np.array(X_train)
# tr[np.isnan(tr)] = 0
np.isnan(tr)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [12]:
from sklearn.preprocessing import StandardScaler

In [13]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [14]:
scaler.transform(tr)

array([[  1.27966051e+00,   1.57460768e+00,   1.42807242e+00, ...,
         -3.38865634e-01,   1.39393754e-01,  -3.63758546e-01],
       [ -2.05478435e-01,  -1.84499121e-02,  -1.30459176e-03, ...,
          1.71699212e-01,  -2.49449948e-01,   3.58643562e-01],
       [ -7.45990850e-03,  -2.25030081e-01,  -2.41743056e-02, ...,
         -3.38865634e-01,  -1.54393281e-01,   1.59944597e+00],
       ..., 
       [  4.75210250e-01,   8.70099094e-01,   1.21856692e+00, ...,
         -4.02686240e-01,  -6.55155371e-01,  -1.60592837e+00],
       [ -1.29458033e+00,  -1.30957601e+00,  -1.26124438e+00, ...,
         -4.02686240e-01,   2.75483603e-01,  -8.67336543e-01],
       [  2.24500083e+00,   2.07289858e+00,   2.22158509e+00, ...,
         -4.02686240e-01,  -1.13496996e+00,   1.56401635e+00]])

In [13]:
# tr = np.array(X_train)
# tr[np.isnan(tr)] = 0
# tr[np.isinf(tr)] = 99
# from sklearn import preprocessing
# tr = preprocessing.scale(tr)
# np.unique(np.isnan(tr)), np.unique(np.isinf(tr))

(array([False], dtype=bool), array([False], dtype=bool))

In [14]:
# from sklearn import preprocessing
# tr = preprocessing.scale(tr)

In [15]:
# tr

array([[  1.27966051e+00,   1.57460768e+00,   1.42807242e+00, ...,
         -3.38865634e-01,   1.39393754e-01,  -3.63758546e-01],
       [ -2.05478435e-01,  -1.84499121e-02,  -1.30459176e-03, ...,
          1.71699212e-01,  -2.49449948e-01,   3.58643562e-01],
       [ -7.45990850e-03,  -2.25030081e-01,  -2.41743056e-02, ...,
         -3.38865634e-01,  -1.54393281e-01,   1.59944597e+00],
       ..., 
       [  4.75210250e-01,   8.70099094e-01,   1.21856692e+00, ...,
         -4.02686240e-01,  -6.55155371e-01,  -1.60592837e+00],
       [ -1.29458033e+00,  -1.30957601e+00,  -1.26124438e+00, ...,
         -4.02686240e-01,   2.75483603e-01,  -8.67336543e-01],
       [  2.24500083e+00,   2.07289858e+00,   2.22158509e+00, ...,
         -4.02686240e-01,  -1.13496996e+00,   1.56401635e+00]])

In [17]:
model.fit(tr, y_train, epochs=30, verbose=2)

Epoch 1/30
 - 204s - loss: 0.1880 - binary_accuracy: 0.9217
Epoch 2/30
 - 202s - loss: 0.1866 - binary_accuracy: 0.9223
Epoch 3/30
 - 209s - loss: 0.1870 - binary_accuracy: 0.9224
Epoch 4/30
 - 211s - loss: 0.1874 - binary_accuracy: 0.9223
Epoch 5/30
 - 210s - loss: 0.1927 - binary_accuracy: 0.9202
Epoch 6/30
 - 210s - loss: 0.1881 - binary_accuracy: 0.9216
Epoch 7/30
 - 214s - loss: 0.1900 - binary_accuracy: 0.9212
Epoch 8/30
 - 215s - loss: 0.1913 - binary_accuracy: 0.9208
Epoch 9/30
 - 215s - loss: 0.1927 - binary_accuracy: 0.9203
Epoch 10/30
 - 219s - loss: 0.1926 - binary_accuracy: 0.9206
Epoch 11/30
 - 219s - loss: 0.1902 - binary_accuracy: 0.9210
Epoch 12/30
 - 219s - loss: 0.1892 - binary_accuracy: 0.9212
Epoch 13/30
 - 217s - loss: 0.1917 - binary_accuracy: 0.9204
Epoch 14/30
 - 217s - loss: 0.1932 - binary_accuracy: 0.9199
Epoch 15/30
 - 218s - loss: 0.1911 - binary_accuracy: 0.9208
Epoch 16/30
 - 218s - loss: 0.1885 - binary_accuracy: 0.9219
Epoch 17/30
 - 221s - loss: 0.191

KeyboardInterrupt: 

In [ ]:
model.predict(t, batch_size=32, verbose=0)

In [ ]:
del X_train

In [ ]:
df_test = pd.read_csv('./data/test.csv')
X_test = pd.read_csv('./test_with_prob.csv')
X_test = X_test.drop('Unnamed: 0', axis=1)
X_test['q1_q2_freq_diff'] = np.abs(X_test['q1_freq'] - X_test['q2_freq'])
X_test['q1_q2_freq_sum'] = X_test['q1_freq'] + X_test['q2_freq']
X_test['skew_diff'] = np.abs(X_test['skew_q1vec'] - X_test['skew_q2vec'])
X_test['skew_sum'] = X_test['skew_q1vec'] + X_train['skew_q2vec']

In [ ]:
X_test.replace(np.inf, 99)
X_test.replace(np.nan, 0)

In [ ]:
scaler.transform

In [ ]:
prob = model.predict(tr, verbose=0， batch_size=4096)

In [ ]:
sub = pd.DataFrame()
sub['test_id'] = df_test['test_id']
sub['is_duplicate'] = prob

In [ ]:
sub.head(10)

In [ ]:
sub.head(10)

In [ ]:

print('Saving :: submission file...')
sub.to_csv('subnn.csv', index=False)
print('\nSubmission result done.')